In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as stats_graph_tsa
import statsmodels.tsa.stattools as stat_tsa_stats
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from pandas_datareader import data as wb
from scipy.stats.distributions import chi2
from datetime import datetime

sns.set()

**Treating Data**

In [9]:
raw_csv_data = pd.read_csv('../01 - Intro/IndexE8.csv')
df_complete = raw_csv_data.copy()
df_complete['market_value'] = df_complete.ftse
df_complete.date = pd.to_datetime(df_complete.date, dayfirst = True)
df_complete.date = df_complete.set_index('date', inplace = True)
df_complete = df_complete.asfreq('b')
df_complete = df_complete.fillna(method = 'ffill')
del df_complete['dax']
del df_complete['ftse']
del df_complete['nikkei']
del df_complete['spx']
size = int(len(df_complete) * 0.8)
df, df_test = df_complete.iloc[:size], df_complete.iloc[size:]

''' Defining the return column '''
df['returns'] = df.market_value.pct_change(1).mul(100)
df = df.iloc[1:]

C:\Users\Yago\AppData\Local\Temp\ipykernel_21928\378421315.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['returns'] = df.market_value.pct_change(1).mul(100)


**AR(1) Simple for Return**

In [11]:
mod_return_ar_1 = ARIMA(df.returns, order = (1, 0, 0))
res_return_ar_1 = mod_return_ar_1.fit()
res_return_ar_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                returns   No. Observations:                 5020
Model:                 ARIMA(1, 0, 0)   Log Likelihood               -7929.749
Date:                Fri, 02 Dec 2022   AIC                          15865.497
Time:                        12:31:24   BIC                          15885.061
Sample:                    01-10-1994   HQIC                         15872.353
                         - 04-05-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0188      0.016      1.143      0.253      -0.013       0.051
ar.L1         -0.0230      0.009     -2.697      0.007      -0.040      -0.006
sigma2         1.3790      0.014    100.092      0.000       1.352       1.406
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):              7872.72
Prob(Q):                              0.93   Prob(JB):                         0.00
Heteroskedasticity (H):               2.04   Skew:                            -0.03
Prob(H) (two-sided):                  0.00   Kurtosis:                         9.13
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""